# Steps taken to clean up the Lang-8 dataset

### Extract raw entries of English learners

In [ ]:
import json
import pandas as pd

with open("lang-8-20111007-2.0/lang-8-20111007-2.0/lang-8-20111007-L1-v2.dat", "r") as f:
    lines = f.readlines()

def clean_control_sequences(s: str) -> str:
    return ''.join(
        c for c in s
        if ord(c) >= 32 or c in '\n\r\t'
    )
data = [json.loads(clean_control_sequences(line)) for line in lines]

rows = []
for entry in data:
    learning_language = entry[2]
    if learning_language != "English":
        continue
    native_language = entry[3]
    learner_sentences = entry[4]
    for sentence in learner_sentences:
        rows.append({'native_language': native_language, 'text': sentence})

df = pd.DataFrame(rows)
df.to_csv("Lang-8/raw/all.csv", index=False)

### Filter any non-english entries

In [ ]:
import pandas as pd
from langdetect import detect

df = pd.read_csv("Lang-8/raw/all.csv")

def is_english_text(text: str) -> bool:
    # check for smaller portions
    words = str(text).split()

    # don't bother for too short text
    if len(words) < 3:
        return False
    
    # check every 3 words
    for i in range(len(words) - 2):
        phrase = ' '.join(words[i:i+3])
        try:
            if detect(phrase) != 'en':
                return False
        except:
            continue
    return True

# split df into english and non-english
english_df = df[df['text'].apply(is_english_text)]
non_english_df = df[~df['text'].apply(is_english_text)]

english_df.to_csv("Lang-8/raw/english_only.csv", index=False)
non_english_df.to_csv("Lang-8/raw/non_english.csv", index=False)

AttributeError: 'float' object has no attribute 'split'